# 에이전트 평가(Evaluating Agents)

이메일 어시스턴트는 라우터를 사용하여 이메일을 분류한 후 응답 생성을 위해 에이전트에게 전달합니다. 실제 운영 환경에서 제대로 작동할지 어떻게 확신할 수 있을까요? 그래서 테스트가 중요한 이유입니다: 응답 품질, 토큰 사용량, 지연 시간, 분류 정확도 같은 정량적 지표를 통해 에이전트 아키텍처에 대한 결정을 안내합니다. 

[LangSmith](https://docs.smith.langchain.com/)는 에이전트 테스트를 위한 두 가지 주요 방법을 제공합니다. 

![overview-img](assets/overview_eval.png)

## Resources

- Notebook Reference: [evaluation.ipynb](https://github.com/langchain-ai/agents-from-scratch/blob/main/notebooks/evaluation.ipynb)
- For running evaluations: [test_tools.py](https://github.com/langchain-ai/agents-from-scratch/blob/main/notebooks/test_tools.py), [/tests/](https://github.com/langchain-ai/agents-from-scratch/tree/main/tests)
- For LangSmith Studio: [src/email_assistant](https://github.com/langchain-ai/agents-from-scratch/tree/main/src/email_assistant)
- Slides: [Building Ambient Agents with LangGraph - Building Agents & Evaluations.pdf](https://files.cdn.thinkific.com/file_uploads/967498/attachments/5f6/a6b/958/Building_Ambient_Agents_with_LangGraph_-_Building_Agents___Evaluations.pdf)

## 환경 설정

In [1]:
from dotenv import load_dotenv


load_dotenv("../../.env", override=True)

True

## 평가 실행 방법

#### Pytest / Vitest

[Pytest](https://docs.pytest.org/en/stable/)와 Vitest는 Python 및 JavaScript 에서 테스트를 작성하기 위한 강력한 도구입니다. LangSmith는 이러한 프레임워크와 통합되어, 결과를 LangSmith에 기록하는 테스트를 작성하고 실행할 수 있도록 지원합니다. 이 노트북에서는 Pytest를 사용합니다.
* Pytest는 해당 프레임워크에 이미 익숙한 개발자가 시작하기에 좋은 방법입니다.
* Pytest는 각 에이전트 테스트 케이스에 일반화하기 어려운 특정 검증과 성공 기준이 필요한 더 복잡한 평가에 유용합니다.

#### LangSmith 데이터셋

LangSmith에서 데이터셋을 생성하고 LangSmith 평가 API를 사용하여 해당 데이터셋에 대해 어시스턴트를 실행할 수 있습니다.
* LangSmith 데이터셋은 테스트 스위트를 협력하여 구축하는 팀에게 유용합니다.
* 운영 트레이스, 어노테이션 큐, 합성 데이터 생성 등을 활용하여 계속해서 확장되는 골든 데이터셋에 예제를 추가할 수 있습니다.
* LangSmith 데이터셋은 데이터셋의 모든 테스트 케이스에 적용할 수 있는 평가자(예: 유사도, 정확히 일치 정확도 등)를 정의할 수 있을 때 특히 유용합니다.

## 테스트 케이스

테스트는 종종 테스트 케이스를 정의하는 것에서 시작하며, 이는 어려운 과정일 수 있습니다. 여기서는 처리하고자 하는 예시 이메일 모음과 테스트할 몇 가지 사항을 `eval/email_dataset.py` 파일에 다음과 같이 정의했습니다.

1.  **Input Emails**: 다양한 이메일 예시 모음
2.  **Ground Truth Classifications**: `Respond`, `Notify`, `Ignore`
3.  **Expected Tool Calls**: 응답이 필요한 각 이메일에 대해 호출되는 도구
4.  **Response Criteria**: 회신이 필요한 이메일에 대한 좋은 응답의 조건

참고로, 우리는 다음과 같은 두 종류의 테스트를 모두 가지고 있습니다.
-   엔드-투-엔드 "통합" 테스트 (예: 입력 이메일 -> 에이전트 -> 최종 결과물 vs 응답 기준)
-   워크플로우의 특정 단계를 위한 테스트 (예: 입력 이메일 -> 에이전트 -> 분류 vs 정답 분류)

In [2]:
from email_assistant.eval.email_dataset import (
    email_inputs,
    expected_tool_calls,
    response_criteria_list,
    triage_outputs_list,
)


test_case_ix = 0

print("Email Input:", email_inputs[test_case_ix])
print("Expected Triage Output:", triage_outputs_list[test_case_ix])
print("Expected Tool Calls:", expected_tool_calls[test_case_ix])
print("Response Criteria:", response_criteria_list[test_case_ix])

Email Input: {'author': '앨리스 스미스 &lt;alice.smith@company.com&gt;', 'to': '랜스 마틴 &lt;lance@company.com&gt;', 'subject': 'API 문서에 대한 간단한 질문', 'email_thread': '안녕하세요, 랜스님,\n\n새로운 인증 서비스의 API 문서를 검토하던 중, 몇몇 엔드포인트가 사양에서 누락된 것을 발견했습니다. 이것이 의도된 것인지, 아니면 문서를 업데이트해야 하는지 명확히 설명해주실 수 있나요?\n\n특히 다음 엔드포인트에 대해 문의드립니다:\n- /auth/refresh\n- /auth/validate\n\n감사합니다!\n앨리스 드림'}
Expected Triage Output: respond
Expected Tool Calls: ['write_email', 'done']
Response Criteria: 
• write_email 도구 호출을 사용하여 이메일을 보내 질문을 인지했음을 알리고 조사가 진행될 것임을 확인합니다.



## Pytest 예제

Pytest를 사용하여 워크플로우의 특정 부분에 대한 테스트를 작성하는 방법을 알아봅니다. 

`email_assistant`가 이메일에 응답할 때 올바른 도구 호출을 하는지 테스트합니다.

In [ ]:
import pytest
from email_assistant.eval.email_dataset import email_inputs, expected_tool_calls
from email_assistant.utils import format_messages_string
from email_assistant.email_assistant import email_assistant
from email_assistant.utils import extract_tool_calls
from langsmith import testing as t


@pytest.mark.langsmith
@pytest.mark.parametrize(
    "email_input, expected_calls",
    [
        # 이메일 답장이 예상되는 몇 가지 예시를 선택하세요
        (email_inputs[0], expected_tool_calls[0]),
        (email_inputs[3], expected_tool_calls[3]),
    ],
)
def test_email_dataset_tool_calls(email_input, expected_calls):
    """
    이메일 처리 과정에서 예상되는 도구 호출이 포함되는지 테스트합니다.

    이 테스트는 이메일 처리 중 모든 예상 도구가 호출되는지 확인하지만,
    도구 호출 순서나 도구별 호출 횟수는 검사하지 않습니다.
    필요 시 이러한 측면에 대한 추가 검사를 포함시킬 수 있습니다.
    """
    # 이메일 어시스턴트를 실행합니다.
    messages = [("user", str(email_input))]
    result = email_assistant.invoke({"messages": messages})

    # 메시지 목록에서 도구 호출 추출
    extracted_tool_calls = extool_calls(result["messages"])
